# Данные

In [1]:
!wget https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv

--2019-06-08 13:37:02--  https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1, 2620:100:6026:1::a27d:4601
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv [переход]
--2019-06-08 13:37:02--  https://www.dropbox.com/s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://ucef8ec168b1b06e78793503c3dc.dl.dropboxusercontent.com/cd/0/inline/AiZI8gFnr2mz4AgVNUNr8BksATY3YToGVdYckdntYCOQkRxIcAwXv-MpObGhO3O6959SYVOTluEAo0yjdV1uJVUyp3qdsEexC8ZFl9seLAqoUA/file# [переход]
--2019-06-08 13:37:03--  https://ucef8ec168b1b06e78793503c3dc.dl.dropboxusercontent.com/cd/0/inline/AiZI8gFnr2mz4AgVNUNr8BksATY3YToGVdYckdntYCOQkRxIcAwXv-MpObGhO3O6959SYVOTluEAo0yjdV1

In [99]:
import collections
import re

import fasttext
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from lightgbm import LGBMClassifier
import nltk
import pymorphy2
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [100]:
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])
df.head()

,text,source
0,Начальник Главного оперативного управления Ген...,mil
1,Артиллерийские подразделения общевойскового об...,mil
2,Подразделения морской пехоты Каспийской флотил...,mil
3,Команды на всеармейских этапах конкурсов АрМИ-...,mil
4,На большом учебно-методическом командирском сб...,mil


# Домашнее задание

В этом домашнем задании вы будете решать задачу тематической классификации. Даны тексты, опубликованные в нескольких пабликах VK.com, посвященных государственным и муниципальным службам. Формально задача заключается в том, чтобы по тексту ($d$) определить в каком паблике он опубликован, то есть, к какому классу $c$ он принадлежит. 

## Задание 1 [1 балл]. Описательные статистики
Посчитайте:
* количество текстов и количество классов
* количество слов (без лемматизации и с лемматизацией) в коллекции
* среднюю длину текста в словах и символах
* найдите 5 самых частых существительных в текстах каждого паблика 

*Рекомендуем использовать pandas для расчета описательных статистик.*

In [101]:
print('Количечтво пустых текстов:', df['text'].isna().sum())
df = df.dropna()

print('Количество текстов с дубликатами:', df.shape[0])
df = df.drop_duplicates()

print('Количество текстов без дубликатов:', df.shape[0])
print('Количество уникальных текстов:', df['text'].nunique())

Количечтво пустых текстов: 99
Количество текстов с дубликатами: 11526
Количество текстов без дубликатов: 11153
Количество уникальных текстов: 11149


In [37]:
print('Количество классов:', df['source'].nunique())
df['source'].value_counts()

Количество классов: 4


mil            2988
mospolice      2792
mchsgov        2784
russianpost    2589
Name: source, dtype: int64

In [38]:
word_collection = set()
df['text'].apply(lambda text: word_collection.update(set(nltk.tokenize.word_tokenize(text))))    
print('Количество слов без лемматизации в коллекции:', len(word_collection))

mystem_analyzer = Mystem()
lemmatize_word_collection = set()
for word in word_collection:
    lemmatize_word_collection.add(mystem_analyzer.lemmatize(word)[0])
    
print('Количество слов c лемматизацией в коллекции:', len(lemmatize_word_collection))

Количество слов без лемматизации в коллекции: 96746
Количество слов c лемматизацией в коллекции: 31489


In [39]:
print('Средняя длина текста в словах:', np.mean([len(nltk.tokenize.word_tokenize(text)) 
                                                for text in df['text']]))
print('Средняя длина текста в символах:', np.mean([len(text) for text in df['text']]))

Средняя длина текста в словах: 122.69765982246929
Средняя длина текста в символах: 711.1856899488927


In [165]:
morph = pymorphy2.MorphAnalyzer()
for source in df['source'].unique():
    counter = collections.Counter()
    for text in df[df['source'] == source]['text']:
        for word in nltk.tokenize.word_tokenize(text): 
            parsed = morph.parse(word)[0]
            if 'NOUN' in parsed.tag:
                counter.update([parsed.normal_form])
    print('Top 5 in', source, '-', ', '.join([word[0] for word in counter.most_common(5)]))

Top 5 in mil - минобороны, военный, россия, год, оборона
Top 5 in mchsgov - мчс, мчсроссия, россия, спасатель, человек
Top 5 in russianpost - почта, россия, год, отделение, письмо
Top 5 in mospolice - полиция, мвд, москва, россия, сотрудник


Разделите коллекцию текстов на обучающую и тестовую части. 

In [102]:
m = Mystem()
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text)).replace('<br>', '')

def lemmatize(text, mystem=m):
    return "".join(m.lemmatize(text)).strip()

df_prep = df.copy()
df_prep.text = df.text.apply(words_only)
df_prep.text = df_prep.text.apply(lemmatize)

X_train, X_test, y_train, y_test = train_test_split(df_prep.drop('source', axis=1), 
                                                    df_prep['source'], test_size=0.2, 
                                                    random_state=42, stratify=df_prep['source'])
del df_prep

 ## Задание 2 [2 балла]. Классификация по правилам
 
 * Разработайте несколько правил вида "Если встречается слово $w$, то текст относится к паблику $c$"
 * Посчитайте, какую точность, полноту, $f$-меру и $accuracy$ вы получаете при классификации по правилам
 * Получилось ли у вас придумать правило, которое никогда не ошибается?

In [264]:
word_sets = {}
len_sets = {}
for source in df['source'].unique():
    word_sets[source] = set()
    len_sets[source] = set()
    for text in df[df['source'] == source]['text']:
        word_sets[source].update(nltk.tokenize.word_tokenize(text))
        len_sets[source].add(len(text))

In [265]:
# запоминаем уникальные для каждого паблика уникальные слова и длины текстов
unique_word_sets = {}
unique_len_sets = {}
for source in word_sets.keys():
    diff_sources = set(word_sets.keys())
    diff_sources.remove(source)
    diff_word_set = set()
    diff_len_set = set()
    for s in diff_sources:
        diff_word_set.update(word_sets[s])
        diff_len_set.update(len_sets[s])
    unique_word_sets[source] = word_sets[source] - diff_word_set
    unique_len_sets[source] = len_sets[source] - diff_len_set

In [266]:
rand = 0
y_pred = []
for text in tqdm.tqdm_notebook(df['text']):
    has_in_sets = False
    tekenized_text = nltk.tokenize.word_tokenize(text)
    for key in unique_word_sets.keys():
        if not has_in_sets:
            if unique_word_sets[key] & set(tekenized_text) or \
                    len(text) in unique_len_sets[key]:
                y_pred.append(key)
                has_in_sets = True
                break
                
    if not has_in_sets:
        rand += 1
        y_pred.append(df['source'].unique()[np.random.randint(0,4)])

In [267]:
print('Случайных ответов:', rand)
np.unique(y_pred, return_counts=True)

Случайных ответов: 304


(array(['mchsgov', 'mil', 'mospolice', 'russianpost'], dtype='<U11'),
 array([2664, 3049, 2817, 2623]))

In [269]:
print('accuracy_score:', accuracy_score(df['source'], y_pred))
print(classification_report(df['source'], y_pred))

accuracy_score: 0.980095041692818
              precision    recall  f1-score   support

     mchsgov       0.99      0.95      0.97      2784
         mil       0.98      1.00      0.99      2988
   mospolice       0.98      0.99      0.98      2792
 russianpost       0.98      0.99      0.98      2589

   micro avg       0.98      0.98      0.98     11153
   macro avg       0.98      0.98      0.98     11153
weighted avg       0.98      0.98      0.98     11153



**При написании правил для всех текстов точность высокая. Падает до 70%, если разбить на трейн-тест**

## Задание 3 [3 балла]. Baseline
Используйте стандартный ```sklearn.pipeline``` для классификации текстов: 
* векторизация 
* $tf-idf$ взвешивание 
* ваш любимый метод классификации.


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [355]:
pipe = Pipeline([('tfidf', TfidfVectorizer(token_pattern=r'\b\w+\b')),
                 ('lgb', LGBMClassifier())])

pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

In [356]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.974002689376961
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.98      0.95      0.97       557
         mil       0.98      0.99      0.98       598
   mospolice       0.98      0.99      0.98       558
 russianpost       0.96      0.97      0.96       518

   micro avg       0.97      0.97      0.97      2231
   macro avg       0.97      0.97      0.97      2231
weighted avg       0.97      0.97      0.97      2231

confusion_matrix:
[[530   7   0  20]
 [  7 590   0   1]
 [  1   2 553   2]
 [  3   2  13 500]]


**Качество очень высокое. 1 и 3, и 2 и 3 классы разделяются лучше всех**

## Задание 4 [2 балла]. Снижение размерности
Добавьте в ваш ```sklearn.pipeline```  методы снижения размерности:  PCA / LSI / LSA / LDA / другое. Какие методы классификации разумно использовать после снижения размерности? Как изменились результаты классификации после добавления нового шага?

In [1]:
pipe = Pipeline([('tfidf', TfidfVectorizer(token_pattern=r'\b\w+\b')),
                 ('svd', TruncatedSVD(n_components=10)),
                 ('lgb', LGBMClassifier())])

pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

NameError: name 'Pipeline' is not defined

In [358]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.9641416405199462
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.97      0.95      0.96       557
         mil       0.98      0.97      0.98       598
   mospolice       0.97      0.99      0.98       558
 russianpost       0.94      0.94      0.94       518

   micro avg       0.96      0.96      0.96      2231
   macro avg       0.96      0.96      0.96      2231
weighted avg       0.96      0.96      0.96      2231

confusion_matrix:
[[529   6   1  21]
 [  7 583   1   7]
 [  3   0 551   4]
 [  8   5  17 488]]


Какие методы классификации разумно использовать после снижения размерности?
- Любые модели, которым нужно меньше признаков
- Любые модели, которые не любят коррелирующие признаки

Как изменились результаты классификации после добавления нового шага?
 - Метрики ухудшиласилс
 - Ошибки предсказания стали более размазаны в матрице ошибок

## Задание 5 [1 балл]. Лемматизация
Посмотрите, как влияет лемматизация на качество классификации. Как изменится качество классификации, если вы используете ```CountVectorizer``` на словах или $n$-граммах на лемматизированных текстах?

In [359]:
pipe = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3), token_pattern=r'\b\w+\b')),
                 ('svd', TruncatedSVD(n_components=10)),
                 ('lgb', LGBMClassifier())])

pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

In [360]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.9605558045719408
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.97      0.92      0.95       557
         mil       0.97      0.97      0.97       598
   mospolice       0.96      0.99      0.97       558
 russianpost       0.94      0.96      0.95       518

   micro avg       0.96      0.96      0.96      2231
   macro avg       0.96      0.96      0.96      2231
weighted avg       0.96      0.96      0.96      2231

confusion_matrix:
[[515  10   5  27]
 [  8 583   3   4]
 [  1   4 550   3]
 [  6   1  16 495]]


In [366]:
pipe = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b')),
                 ('lgb', LGBMClassifier())])

pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

In [367]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.9748991483639623
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.98      0.96      0.97       557
         mil       0.98      0.99      0.98       598
   mospolice       0.98      0.99      0.98       558
 russianpost       0.96      0.96      0.96       518

   micro avg       0.97      0.97      0.97      2231
   macro avg       0.97      0.97      0.97      2231
weighted avg       0.97      0.97      0.97      2231

confusion_matrix:
[[532   7   1  17]
 [  7 591   0   0]
 [  0   2 554   2]
 [  4   4  12 498]]


**При использовании n-грамм без использования понижения размерности качество выросло**

## Задание 6 [3 балла]. Continious bag of words
Для baseline решения мы использовали обычное представление текста в виде мешка слов. Попробуйте использовать другие модели представления текста – например, в виде непрерывного мешка слов, то есть, в виде набора эмбеддингов. Для того, чтобы получить вектор текста попробуйте:
* усреднить все эмбеддинги слов, входящих в этот текст
* усреднить все эмбеддинги слов, входящих в этот текст с $tf-idf$ весами
* использовать любую модель эмбеддинга документа.

Используйте любую модель эмбеддингов по вашему вкусу. 


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [36]:
texts = [pd.concat([X_train, X_test])['text'].iloc[i].split() for i in range(len(df))]

In [38]:
w2v_model = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)
tfidf = TfidfVectorizer(token_pattern=r'\b\w+\b')
tfidf.fit(X_train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='\\b\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
def get_emb(data, tfidf=None):
    emb = []
    for text in data['text']:
        text_emb = []
        for word in text:
            try:
                if tfidf is None:
                    text_emb.append(w2v_model.wv.get_vector(word))
                else:
                    text_emb.append(w2v_model.wv.get_vector(word) * tfidf.vocabulary_[word])
            except KeyError:
                pass
        emb.append(np.mean(text_emb, axis=0))
    return emb

In [377]:
train_emb = get_emb(X_train)
test_emb = get_emb(X_test)

In [379]:
pipe = Pipeline([('lgb', LGBMClassifier())])

pipe.fit(train_emb, y_train)
y_pred = pipe.predict(test_emb)

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.789332138054684
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.86      0.77      0.81       557
         mil       0.80      0.84      0.82       598
   mospolice       0.81      0.80      0.81       558
 russianpost       0.69      0.73      0.71       518

   micro avg       0.79      0.79      0.79      2231
   macro avg       0.79      0.79      0.79      2231
weighted avg       0.79      0.79      0.79      2231

confusion_matrix:
[[430  29  23  75]
 [ 22 503  32  41]
 [ 16  39 448  55]
 [ 31  55  52 380]]


In [8]:
train_emb = get_emb(X_train, tfidf=tfidf)
test_emb = get_emb(X_test, tfidf=tfidf)

In [9]:
pipe = Pipeline([('lgb', LGBMClassifier())])

pipe.fit(train_emb, y_train)
y_pred = pipe.predict(test_emb)

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.7785746302106679
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.86      0.78      0.82       557
         mil       0.79      0.80      0.80       598
   mospolice       0.78      0.78      0.78       558
 russianpost       0.69      0.75      0.72       518

   micro avg       0.78      0.78      0.78      2231
   macro avg       0.78      0.78      0.78      2231
weighted avg       0.78      0.78      0.78      2231

confusion_matrix:
[[434  24  24  75]
 [ 22 478  55  43]
 [ 13  57 434  54]
 [ 36  45  46 391]]


In [25]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=5, workers=4)

In [26]:
train_vec = []
for text in texts[:X_train.shape[0]]:
    train_vec.append(model.infer_vector(texts[0]))
test_vec = []
for text in texts[X_train.shape[0]:]:
    test_vec.append(model.infer_vector(texts[0]))

In [27]:
pipe = Pipeline([('lgb', LGBMClassifier())])

pipe.fit(train_vec, y_train)
y_pred = pipe.predict(test_vec)

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.2478709099058718
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.24      0.23      0.23       557
         mil       0.25      0.33      0.29       598
   mospolice       0.25      0.25      0.25       558
 russianpost       0.24      0.17      0.20       518

   micro avg       0.25      0.25      0.25      2231
   macro avg       0.25      0.25      0.24      2231
weighted avg       0.25      0.25      0.24      2231

confusion_matrix:
[[126 210 129  92]
 [151 199 154  94]
 [122 201 138  97]
 [132 174 122  90]]


**Наблюдения**
 - Doc2Vec показал очень плохие результаты, результаты случайны
 - Word2Vec показал результаты хуже обычного tf-idf
 - Word2Vec * tf-idf результат слегка ухудшился

## Задание 7 [2 балла]. fastText

Используйте ```fastText``` в режиме классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [23]:
rows = []
for idx, text in enumerate(X_train['text']):
    rows.append(text + ' __label__' + y_train.iloc[idx])

np.savetxt('data.train.txt', rows, fmt='%s')

rows = []
for idx, text in enumerate(X_test['text']):
    rows.append(text)

np.savetxt('data.test.txt', rows, fmt='%s')

In [32]:
classifier = fasttext.supervised('data.train.txt', 'model')
y_pred = [cls[0] for cls in classifier.predict(X_test['text'])]

In [34]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.9668310174809502
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.97      0.95      0.96       557
         mil       0.98      0.98      0.98       598
   mospolice       0.97      0.98      0.97       558
 russianpost       0.94      0.96      0.95       518

   micro avg       0.97      0.97      0.97      2231
   macro avg       0.97      0.97      0.97      2231
weighted avg       0.97      0.97      0.97      2231

confusion_matrix:
[[528   7   2  20]
 [  9 587   0   2]
 [  1   2 546   9]
 [  4   3  15 496]]


**Наблюдения**
- результат FastText сравним с LGBM на tf-idf, но в разы быстрее и требует меньше памяти

## Задание 8 [4 балла]. CNN

Реализуйте модель Kim et al (2014) для решения задачи классификации с помощью CNN. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).
Ссылка: Kim Y. Convolutional Neural Networks for Sentence Classification. 2014

In [118]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.utils import to_categorical

batch_size = 64
num_epochs = 10
emb_size = 100
sequence_length = 50


def build_model(w2v_model):
    model_input = Input(shape=(sequence_length,))
    x = Embedding(len(w2v_model.wv.vocab), emb_size, input_length=sequence_length, name="embedding")(model_input)
    x = Dropout(0.2)(x)

    conv_blocks = []
    for sz in [3, 7]:
        conv = Convolution1D(filters=10, kernel_size=sz,  padding="valid", activation="relu", strides=1)(x)
        conv = MaxPooling1D()(conv)
        conv = Flatten()(conv)
        conv_blocks.append(conv)

    x = Concatenate()(conv_blocks)

    x = Dropout(0.3)(x)
    x = Dense(50, activation="relu")(x)
    model_output = Dense(4, activation="softmax")(x)

    model = Model(model_input, model_output)
    
    # Initialize weights with word2vec
    weights = np.array([w2v_model.wv.get_vector(word) for word in w2v_model.wv.vocab.keys()])
    embedding_layer = model.get_layer("embedding")
    embedding_layer.set_weights([weights])
    return model


w2v_model = Word2Vec(texts, size=emb_size, window=5, min_count=1, workers=4)
model = build_model(w2v_model)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [119]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 100)      3550500     input_15[0][0]                   
__________________________________________________________________________________________________
dropout_25 (Dropout)            (None, 50, 100)      0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 48, 10)       3010        dropout_25[0][0]                 
__________________________________________________________________________________________________
conv1d_26 

In [120]:
word_to_idx = {key:idx for idx, key in enumerate(w2v_model.wv.vocab.keys())}

In [96]:
x_train = sequence.pad_sequences([[word_to_idx[word] for word in text.split()] for text in X_train['text']], 
                                   maxlen=sequence_length, padding="post", truncating="post")
x_test = sequence.pad_sequences([[word_to_idx[word] for word in text.split()] for text in X_test['text']], 
                                  maxlen=sequence_length, padding="post", truncating="post")

In [124]:
labels = {'mchsgov': 0, 'mil': 1, 'mospolice': 2, 'russianpost': 3}
inv_labels = {0: 'mchsgov', 1: 'mil', 2: 'mospolice', 3: 'russianpost'}

In [121]:
model.fit(x_train, to_categorical([labels[y] for y in y_train], num_classes=4),
          validation_data=(x_test, to_categorical([labels[y] for y in y_test], num_classes=4),),
          batch_size=batch_size, epochs=num_epochs, verbose=2)

Train on 8922 samples, validate on 2231 samples
Epoch 1/10
 - 11s - loss: 0.1988 - acc: 0.9255 - val_loss: 0.0963 - val_acc: 0.9752
Epoch 2/10
 - 10s - loss: 0.0932 - acc: 0.9729 - val_loss: 0.0772 - val_acc: 0.9804
Epoch 3/10
 - 10s - loss: 0.0731 - acc: 0.9809 - val_loss: 0.0707 - val_acc: 0.9811
Epoch 4/10
 - 11s - loss: 0.0640 - acc: 0.9822 - val_loss: 0.0706 - val_acc: 0.9818
Epoch 5/10
 - 11s - loss: 0.0548 - acc: 0.9843 - val_loss: 0.0668 - val_acc: 0.9820
Epoch 6/10
 - 11s - loss: 0.0495 - acc: 0.9852 - val_loss: 0.0680 - val_acc: 0.9821
Epoch 7/10
 - 11s - loss: 0.0428 - acc: 0.9873 - val_loss: 0.0646 - val_acc: 0.9842
Epoch 8/10
 - 11s - loss: 0.0361 - acc: 0.9888 - val_loss: 0.0649 - val_acc: 0.9835
Epoch 9/10
 - 11s - loss: 0.0335 - acc: 0.9893 - val_loss: 0.0732 - val_acc: 0.9820
Epoch 10/10
 - 11s - loss: 0.0299 - acc: 0.9906 - val_loss: 0.0662 - val_acc: 0.9827


In [126]:
y_pred = [inv_labels[y] for y in np.argmax(model.predict(x_test), axis=1)]

In [128]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:', classification_report(y_test, y_pred), sep='\n')
print('confusion_matrix:', confusion_matrix(y_test, y_pred), sep='\n')

accuracy_score: 0.9645898700134469
classification_report:
              precision    recall  f1-score   support

     mchsgov       0.96      0.95      0.96       557
         mil       0.99      0.98      0.98       598
   mospolice       0.96      0.99      0.98       558
 russianpost       0.95      0.93      0.94       518

   micro avg       0.96      0.96      0.96      2231
   macro avg       0.96      0.96      0.96      2231
weighted avg       0.96      0.96      0.96      2231

confusion_matrix:
[[531   3   1  22]
 [  7 584   3   4]
 [  4   0 553   1]
 [ 12   3  19 484]]


**Наблюдения**
- Быстрый рост метрик и высокая точность
- Результат лучше, чем у FastText
- Можно улучшить перебирая гиперпараметры

## Задание 9 [4 + 2 балла]. RNN

(4 балла)Используйте ```RNN``` (BLSTM с какими-то признаками и пулинг поверх) для решения задачи текстовой классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).

За дополнительные 2 балла добавьте в модель символьные признаки - CharCNN или CharRNN

In [158]:
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional, Input, concatenate
from keras.models import Model

w2v_model = Word2Vec(texts, size=emb_size, window=5, min_count=1, workers=4)

batch_size = 64
num_epochs = 10
emb_size = 100
sequence_length = 50

def build_model(w2v_model):
    weights = np.array([w2v_model.wv.get_vector(word) for word in w2v_model.wv.vocab.keys()])
    
    # Вход для BiLSTM
    model_input_sent = Input(shape=(sequence_length,))
    x = Embedding(len(w2v_model.wv.vocab), emb_size, 
                  input_length=sequence_length, weights=[weights], 
                  name="embedding")(model_input_sent)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = MaxPooling1D()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(4, activation="softmax")(x)
    model = Model(inputs=[model_input_sent], outputs=x)
    return model

In [159]:
model = build_model(w2v_model)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 50)                0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           3550500   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 50, 64)            34048     
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 25, 64)            0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 25, 64)            0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 6404      
Total para

In [160]:
model.fit(x_train, to_categorical([labels[y] for y in y_train], num_classes=4),
          validation_data=(x_test, to_categorical([labels[y] for y in y_test], num_classes=4)),
          batch_size=batch_size, epochs=num_epochs, verbose=2)

Train on 8922 samples, validate on 2231 samples
Epoch 1/10
 - 17s - loss: 0.2881 - acc: 0.9147 - val_loss: 0.1681 - val_acc: 0.9534
Epoch 2/10
 - 15s - loss: 0.1237 - acc: 0.9668 - val_loss: 0.1333 - val_acc: 0.9623
Epoch 3/10
 - 14s - loss: 0.0853 - acc: 0.9770 - val_loss: 0.1219 - val_acc: 0.9677
Epoch 4/10
 - 14s - loss: 0.0611 - acc: 0.9831 - val_loss: 0.1240 - val_acc: 0.9615
Epoch 5/10
 - 14s - loss: 0.0412 - acc: 0.9890 - val_loss: 0.1281 - val_acc: 0.9700
Epoch 6/10
 - 14s - loss: 0.0295 - acc: 0.9920 - val_loss: 0.1513 - val_acc: 0.9659
Epoch 7/10
 - 14s - loss: 0.0205 - acc: 0.9941 - val_loss: 0.1569 - val_acc: 0.9718
Epoch 8/10
 - 14s - loss: 0.0162 - acc: 0.9953 - val_loss: 0.1488 - val_acc: 0.9718
Epoch 9/10
 - 15s - loss: 0.0150 - acc: 0.9957 - val_loss: 0.1678 - val_acc: 0.9704
Epoch 10/10
 - 15s - loss: 0.0098 - acc: 0.9969 - val_loss: 0.1806 - val_acc: 0.9709


**Наблюдения**
- LSTM работает не хуже, чем CNN

## Задание 10 [8 баллов]. ULMFit

Используйте ```ULMFit``` для решения задачи классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Конец
Выполните какие-то из предыдущих заданий. Для всех заданий, кроме задания 1 требуется вычислить метрику accuracy метода.

Подведите итоги и проведите сравнение всех использованных методов. Какой из них показался вам лучше и почему?

**NB!** Задание обязательное вне зависимости от того, сколько из предыдущих пунктов вы выполнили, и дополнительных баллов не дает.


Для получения полной оценки за NLP-часть достаточно набрать **20 баллов**.

**Итоги**
- Частные сравнения описанны в наблюдениях к каждому заданию.
- Лучший из методов с точки зрения простоты - tfidf + classifier. Но она скорее всего будет показывать плохой результат на более сложных данных
- Следущий по простоте и лучший по качеству CNN с предобученными эмбэдингами от w2v, которые сами по себе дают плохой результат в связке со стандартным классификатором

# Правила сдачи 

1. Домашняя работа должна быть выполнена в ipynb-тетрадке.
2. Сделанную тетрадку нужно отправить ассистенту (ссылка на контакты будет в вики).
3. Задание выполняется индивидуально.
4. Все вычисления должны быть снабжены пояснениями!
5. Дедлайн – 10 июня в 10.00.
